# Analizando tuits

[Pablo Haya](http://pablohaya.com), Ultima versión: Mar-2018

## Objetivo

Este _notebook_ presenta analisis y visualizaciones básicas sobre el conjunto de tuits descargados en la práctica anterior.

### Inicialización

Primeramente se cargan las bibliotecas necesarias para realizar el análisis de datos, `pandas`, y para la visualización `matplotlib`. Se añaden también bibliotecas auxiliares para procesar el formato JSON, y para poder utilizar expresiones regulares `re`.

In [ ]:
# Bibliotecas necesarias
import json
import matplotlib.pyplot as plt
import pandas as pd
import re

### Análisis

Leemos los tuits descargados en la práctica anterior. En el caso de que se haya tenido alguna dificultad se puede utilizar el conjunto de datos de ejemplo que acompaña a este práctica 

In [ ]:
# Lee el dataset descargado anteriormente
# Es preciso incluir el nombre del fichero correcto
tweets_data_path = 'twitter_stream_data.json'

tweets_data = []
with open(tweets_data_path, "r") as tweets_file:
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue

In [ ]:
# Imprimimos cuantos tweets hay
print(len(tweets_data))

Cada tuit se describe mediante el formato JSON como vamos a poder ver a continuación imprimiendo el primer tuit descargado

In [ ]:
print(json.dumps(tweets_data[0], indent=4))

Como se puede comprobar, cada tuit incluye un gran cantidad de metainformación adicional que se puede explotar a parte del texto. La descripción completa de los campos de un tuit se puede encontrar en: https://dev.twitter.com/overview/api/tweets.

El siguiente ejemplo muestra la estructura de un tuit reflejando los campos más importantes: 

```json
{
  "created_at": "Tue Apr 03 06:34:43 +0000 2018",
  "id": 981057390220017700,
  "id_str": "981057390220017664",
  "text": "Hadoop training done know what are the career opportunities to utilize https://t.co/uqUxnZIm76",
  "favorite_count":, 
  "retweet_count":,
  "user": {...},
  "entities": {...},
  "lang": "en",
  "coordinates": {...}, 
  "place": {...}
}
```

`id` e `id_str` identifican univocamente un tuit que ha sido creado en la fecha indicada por el atributo `created_at`. 

Los atributos `favorited_count` y `retweet_count`indican aproximadamente el número de favoritos y retuits que ha recibido el tuit hasta el momento. Es posible conseguir el valor actualizado de estos atributos preguntando de nuevo a la API especificando el identificador del tuit como parámetro de la función `get_status`.

El atributo `user` incluye una estructura con información detallada sobre el usuario que ha emitido el tuit como el número de seguidores y seguidos, si es una cuenta verificada, o el número de tuits entre otros. En el campo `entities` podemos encontrar la lista de hashtags incluidos en el tuit o las URLS entre otros.

Los tuits geolocalizados incluyen información de la posición en dos niveles. Dentro del objeto `coordinates` podemos encontrar la coordenadas exactas (longitud y latitud), mientras que el objeto `place` representa la localiación como un región ya sea mediante un etiqueta, como puede ser el pais o el nombre de la ciudad, o mediante un polígono. La información de localización es opcional, por lo que no está presente en todos los tuits. Puede ser que se disponga del  lugar pero no de la localizacióne exacta, pero no al revés.

In [ ]:
# Creamos un data frame
tweets = pd.DataFrame()

# Extraemos el texto, idioma, y pais de cada tuit, los almacenamos columnas distintas del data frame
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['lang'] = list(map(lambda tweet: tweet['lang'], tweets_data))
tweets['country'] = list(map(lambda tweet: tweet['place']['country'] if tweet['place'] != None else None, tweets_data))

### ¿Cuales son los cinco idiomas más usados en el dataset?

Calculamos el agregado de tuits por lenguaje, y lo visualizamos

In [ ]:
tweets_by_lang = tweets['lang'].value_counts()
tweets_by_lang

In [ ]:
# y los visualizamos
fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Idioma', fontsize=15)
ax.set_ylabel('Número de tweets' , fontsize=15)
ax.set_title('5 idiomas más usados', fontsize=15, fontweight='bold')
tweets_by_lang[:5].plot(ax=ax, kind='bar', color='red')
plt.show()

### ¿Cuales son los cinco países desde donde se ha tuiteado más?

Primero calculamos el porcentaje de tuits que tienen localización asociada respecto al total, el cual suele ser muy bajo

In [ ]:
tweets_by_country = tweets['country'].value_counts()
print("{:.1%}".format(sum(tweets_by_country) / len(tweets['country'])))

Mostramos el agregado de tuits por pais y los visualizamos

In [ ]:
tweets_by_country

In [ ]:
# y los visualizamos
fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Idiomas', fontsize=15)
ax.set_ylabel('Numero de tutis' , fontsize=15)
ax.set_title('5 paises más presentes', fontsize=15, fontweight='bold')
tweets_by_country[:5].plot(ax=ax, kind='bar', color='red')
plt.show()

### ¿Cuál de las dos tecnologías ha sido más mencionada?

Añadimos una columna adicional por cada término que indican si aparece o no ese término en el tuit. A continuación agregamos el número de apariciones en cada columna.

In [ ]:
# Busca un término en un texto
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

# Añade una columna al dataframe tweets 
tweets['hadoop'] = tweets['text'].apply(lambda tweet: word_in_text('hadoop', tweet))
tweets['spark'] = tweets['text'].apply(lambda tweet: word_in_text('spark', tweet))

# Calculamos el número de tweets por cada tecnología big data
print('n hadoop:', tweets['hadoop'].value_counts()[True])
print('n spark', tweets['spark'].value_counts()[True])

In [ ]:
# Comparación entre las distintas tecnologías
big_tech = ['hadoop', 'spark']
tweets_by_big_tech = [tweets['hadoop'].value_counts()[True], tweets['spark'].value_counts()[True]]

x_pos = list(range(len(big_tech)))
width = 0.8
fig, ax = plt.subplots()
plt.bar(x_pos, tweets_by_big_tech, width, alpha=1, color='r')

ax.set_ylabel('Número de tuits', fontsize=15)
ax.set_title('hadoop vs spark', fontsize=15, fontweight='bold')
ax.set_xticks([p + 0.05 * width for p in x_pos])
ax.set_xticklabels(big_tech)
plt.show()

### ¿Cual de las dos tecnologías ha compartido más URL?

In [ ]:
# Extramos las urls para hayar el número que hay en el dataset
# Extraer url
regex = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' # URLs
url_re = re.compile(regex)
    
def extract_urls(s):
    return url_re.findall(s)
    
tweets['url'] = tweets['text'].apply(lambda tweet: extract_urls(tweet))

print('hadoop:',tweets[tweets['hadoop'] == True]['url'].apply(lambda url: True if url else False).value_counts()[True])
print('spark:',tweets[tweets['spark'] == True]['url'].apply(lambda url: True if url else False).value_counts()[True])

### Ejercicios

1. Dibujar un gráfico que compare las urls más compartidas de ambas tecnologías
- Calcular los 5 usuarios más activos